## Session-Based Recommendation


* 세션 데이터를 기반으로 유저가 다음에 클릭 또는 구매할 아이템을 예측하는 추천
  * 쿠키: 로컬에 저장되는 사용자 정보(방문정보, 장바구니 정보 등)
  * 세션: 웹에 저장되는 방문정보(로그인 정보, 구매정보 등)
  * 캐시: 로딩 비용을 줄이기 위해 임시 저장한 데이터
  
  
* 단점: 사용자의 profile이 없는 상태에서의 잠재정보에 한정된 Latent Vector를 사용하는 것이 어려움


* 대안: 사용자의 순서정보를 이용한 추천시스템(RNN기반)
  * session state: actual event(one-hot-vector), events in the session(가중치 합계)

### Sequential Recommendation: 유저의 정보를 알 수 있는 데이터
* 유저와 아이템의 추가 정보를 Sequential Recommendation 모델에 적용하는 분야는 Context-Aware 라는 키워드로 활발히 연구되고 있습니다.



## 데이터 불러오기

In [1]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
import datetime
warnings.filterwarnings('ignore')

In [2]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose-data/ml-1m') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


여기서 이전 실습내역과 가장 크게 다른 부분은 바로 SessionID 대신 UserID 항목이 들어갔다는 점입니다. 이 데이터셋은 명확한 1회 세션의 SessionID를 포함하지 않고 있습니다. 그래서 이번에는 UserID가 SessionID 역할을 해야 합니다.

Rating 정보가 포함되어 있습니다. 이전 실습내역에서는 이런 항목이 포함되어 있지 않았으므로, 무시하고 제외할 수 있습니다. 하지만, 직전에 봤던 영화가 맘에 들었는지 여부가 비슷한 영화를 더 고르게 하는 것과 상관이 있을 수도 있습니다. 아울러, Rating이 낮은 데이터를 어떻게 처리할지도 고민해야 합니다.

Time 항목에는 UTC time 가 포함되어, 1970년 1월 1일부터 경과된 초단위 시간이 기재되어 있습니다.

### Step 1. 데이터의 전처리
위와 같이 간단히 구성해 본 데이터셋을 꼼꼼이 살펴보면서 항목별 기본분석, session length, session time, cleaning 등의 작업을 진행합니다.
특히, 이 데이터셋에서는 Session이 아닌 UserID 단위로 데이터가 생성되어 있으므로, 이를 Session 단위로 어떻게 해석할지에 주의합니다.

추천시스템을 구축할 때 가장 먼저 확인해 볼 것은 유저수(세션 수)와 아이템 수 입니다.

In [3]:
data.nunique()

UserId      6040
ItemId      3706
Rating         5
Time      458455
dtype: int64

### 2.2 Session Length
* 같은 SessionId를 공유하는 데이터 row의 개수
* 로그인하지 않았기 때문에 이 사용자를 특정할 수 없기에, 특정 사용자의 행동을 SessionId 기준으로 모아서(개별 아이디로 판단) 분류

In [4]:
session_length = data.groupby(['UserId']).size()
session_length

UserId
1        53
2       129
3        51
4        21
5       198
       ... 
6036    888
6037    202
6038     20
6039    123
6040    341
Length: 6040, dtype: int64

In [5]:
session_length.median(), session_length.mean()

(96.0, 165.5975165562914)

In [6]:
session_length.min(), session_length.max()

(20, 2314)

In [7]:
session_length.quantile(q=0.999)

1343.181000000005

In [8]:
long_session = session_length[session_length == 2314].index[0]
data[data['UserId']==long_session]

,UserId,ItemId,Rating,Time
696969,4169,1268,5,965333392
697168,4169,2617,4,965333392
697185,4169,2628,4,965333392
697219,4169,2653,4,965333392
697275,4169,423,3,965333392
...,...,...,...,...
697882,4169,3754,2,1024174347
695702,4169,1413,3,1024175031
697358,4169,494,4,1024175760
695945,4169,1804,2,1024175783


In [56]:
len(data)

1000209

#### 세션길이 기준 하위 99.9%까지의 분포 누적합을 시각화

In [9]:
length_count = session_length.groupby(session_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

length_percent_cumsum_999

20      0.014238
21      0.029305
22      0.042053
23      0.055464
24      0.068874
          ...   
1271    0.998179
1277    0.998344
1286    0.998510
1302    0.998675
1323    0.998841
Length: 736, dtype: float64

In [10]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(20, 10))
# plt.bar(x=length_percent_cumsum_999.index,
#         height=length_percent_cumsum_999, color='red')
# plt.xticks(length_percent_cumsum_999.index)
# plt.yticks(np.arange(0, 1.01, 0.05))
# plt.title('Cumsum Percentage Until 0.999', size=20)
# plt.show()

#### Session Time


유저 클릭수의 이상 여부를 확인하기 위해 시간대별로 확인

In [11]:
type(data['Time'])

pandas.core.series.Series

In [12]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

956703932
1046454590


In [13]:
type(latest)

int

In [14]:
import time
time.localtime(oldest)

time.struct_time(tm_year=2000, tm_mon=4, tm_mday=26, tm_hour=8, tm_min=5, tm_sec=32, tm_wday=2, tm_yday=117, tm_isdst=0)

In [15]:
a = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(oldest))
print(a)
print(type(a))

2000-04-26 08:05:32
<class 'str'>


In [16]:
b = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(latest))
b

'2003-03-01 02:49:50'

In [17]:
data_1 = data.copy()
data_1

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [18]:
x = [time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime(c_time)) for c_time in data_1['Time']]
x[:10]

['2001-01-01-07:00:19',
 '2001-01-01-07:00:55',
 '2001-01-01-07:00:55',
 '2001-01-01-07:00:55',
 '2001-01-01-07:01:43',
 '2001-01-01-07:02:52',
 '2001-01-01-07:04:35',
 '2001-01-01-07:11:59',
 '2001-01-01-07:11:59',
 '2001-01-01-07:12:40']

In [19]:
data_1['newTime'] = [time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime(c_time)) for c_time in data_1['Time']]
data_1

,UserId,ItemId,Rating,Time,newTime
31,1,3186,4,978300019,2001-01-01-07:00:19
22,1,1270,5,978300055,2001-01-01-07:00:55
27,1,1721,4,978300055,2001-01-01-07:00:55
37,1,1022,5,978300055,2001-01-01-07:00:55
24,1,2340,3,978300103,2001-01-01-07:01:43
...,...,...,...,...,...
1000019,6040,2917,4,997454429,2001-08-10-23:40:29
999988,6040,1921,4,997454464,2001-08-10-23:41:04
1000172,6040,1784,3,997454464,2001-08-10-23:41:04
1000167,6040,161,3,997454486,2001-08-10-23:41:26


In [20]:
k = data_1['newTime']
k

31         2001-01-01-07:00:19
22         2001-01-01-07:00:55
27         2001-01-01-07:00:55
37         2001-01-01-07:00:55
24         2001-01-01-07:01:43
                  ...         
1000019    2001-08-10-23:40:29
999988     2001-08-10-23:41:04
1000172    2001-08-10-23:41:04
1000167    2001-08-10-23:41:26
1000042    2001-08-20-22:44:15
Name: newTime, Length: 1000209, dtype: object

In [21]:
result = []
for i in range(len(k)):
    if k[i][:4] == '2003':
        result.append(k[i])
    elif k[i][:4] == '2002':
        result.append(k[i])
result[:3]

['2002-03-12-12:58:05', '2002-03-12-12:52:07', '2002-02-03-12:41:37']

In [22]:
len(result)

27436

In [23]:
df = data_1.loc[data_1['newTime'].isin(result), :]
df

,UserId,ItemId,Rating,Time,newTime
5125,36,1292,5,1012707635,2002-02-03-12:40:35
5053,36,7,4,1012707697,2002-02-03-12:41:37
5170,36,1387,5,1015904819,2002-03-12-12:46:59
5267,36,1201,4,1015904819,2002-03-12-12:46:59
5122,36,1291,5,1015904836,2002-03-12-12:47:16
...,...,...,...,...,...
994100,6002,2013,4,1014524679,2002-02-24-13:24:39
993890,6002,2520,4,1014524680,2002-02-24-13:24:40
994045,6002,1387,5,1014524720,2002-02-24-13:25:20
993900,6002,1927,4,1014524758,2002-02-24-13:25:58


In [24]:
df.isna().sum()

UserId     0
ItemId     0
Rating     0
Time       0
newTime    0
dtype: int64

In [25]:
user_min = data.groupby(['UserId']).size().min()
user_min

20

아이디당 세션의 경우 최소 1 이상이어야 다음 선택을 예측할 수 있는데 본 데이터는 최소 클릭 수가 20회이기 때문에 클렌징 생략

In [26]:
# # short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# # 이를 위해 반복문을 통해 지속적으로 제거 합니다.
# def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
#     while True:
#         before_len = len(data)
#         data = cleanse_short_session(data, shortest)
#         data = cleanse_unpopular_item(data, least_click)
#         after_len = len(data)
#         if before_len == after_len:
#             break
#     return data


# def cleanse_short_session(data: pd.DataFrame, shortest):
#     session_len = data.groupby('SessionId').size()
#     session_use = session_len[session_len >= shortest].index
#     data = data[data['SessionId'].isin(session_use)]
#     return data


# def cleanse_unpopular_item(data: pd.DataFrame, least_click):
#     item_popular = data.groupby('ItemId').size()
#     item_use = item_popular[item_popular >= least_click].index
#     data = data[data['ItemId'].isin(item_use)]
#     return data

In [27]:
# data = cleanse_recursive(data, shortest=2, least_click=5)
# data

In [28]:
type(df['newTime'])

pandas.core.series.Series

In [29]:
df['newTime'][:1]

5125    2002-02-03-12:40:35
Name: newTime, dtype: object

In [30]:
# l = datetime.datetime.strptime('2002-12-22', '%Y-%m-%d').date()
# print(type(l))

In [31]:
result_1 = []
for i in df['newTime']:
    result_1.append(datetime.datetime.strptime(i, '%Y-%m-%d-%H:%M:%S').date())
result_1[:3]

[datetime.date(2002, 2, 3),
 datetime.date(2002, 2, 3),
 datetime.date(2002, 3, 12)]

In [32]:
df['newTime'] = result_1
df

,UserId,ItemId,Rating,Time,newTime
5125,36,1292,5,1012707635,2002-02-03
5053,36,7,4,1012707697,2002-02-03
5170,36,1387,5,1015904819,2002-03-12
5267,36,1201,4,1015904819,2002-03-12
5122,36,1291,5,1015904836,2002-03-12
...,...,...,...,...,...
994100,6002,2013,4,1014524679,2002-02-24
993890,6002,2520,4,1014524680,2002-02-24
994045,6002,1387,5,1014524720,2002-02-24
993900,6002,1927,4,1014524758,2002-02-24


In [33]:
final_time = df['newTime'].max()
print(final_time)
session_last_time = df.groupby('UserId')['newTime'].max()
print(session_last_time)

2003-03-01
UserId
36      2002-12-22
59      2003-01-08
65      2002-12-29
102     2003-01-29
104     2002-03-14
           ...    
5950    2003-02-28
5956    2002-09-30
5991    2002-01-13
5996    2002-11-05
6002    2002-02-24
Name: newTime, Length: 587, dtype: object


In [34]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['newTime'].max()
    print(final_time)
    session_last_time = data.groupby('UserId')['newTime'].max()
    session_in_train = session_last_time[session_last_time < final_time - dt.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - dt.timedelta(n_days)].index

    before_date = data[data['UserId'].isin(session_in_train)]
    after_date = data[data['UserId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

In [35]:
tr, test = split_by_date(df, n_days=30)
tr, val = split_by_date(tr, n_days=30)

2003-03-01
2003-01-29


In [36]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["newTime"].min()}\n'
          f'\t Last Time : {data["newTime"].max()}\n')

In [37]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 14888
	 Sessions: 402
	 Items: 2632
	 First Time : 2002-01-01
	 Last Time : 2002-12-29

* valid Set Stats Info
	 Events: 3474
	 Sessions: 82
	 Items: 1510
	 First Time : 2002-01-01
	 Last Time : 2003-01-29

* test Set Stats Info
	 Events: 8533
	 Sessions: 103
	 Items: 2259
	 First Time : 2002-01-01
	 Last Time : 2003-03-01



In [55]:
len(df)

27436

In [38]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [39]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 14888
	 Sessions: 402
	 Items: 2632
	 First Time : 2002-01-01
	 Last Time : 2002-12-29

* valid Set Stats Info
	 Events: 3474
	 Sessions: 82
	 Items: 1510
	 First Time : 2002-01-01
	 Last Time : 2003-01-29

* test Set Stats Info
	 Events: 8533
	 Sessions: 103
	 Items: 2259
	 First Time : 2002-01-01
	 Last Time : 2003-03-01



In [57]:
# from datetime import timedelta
# # Default: days
# timedelta(seconds = latest)-timedelta(days = 30)

datetime.timedelta(days=12081, seconds=64190)

### Step 2. 미니 배치의 구성
실습코드 내역을 참고하여 데이터셋과 미니 배치를 구성해 봅시다. Session-Parallel Mini-Batch의 개념에 따라, 학습 속도의 저하가 최소화될 수 있도록 구성합니다.
단, 위 Step 1에서 Session 단위를 어떻게 정의했느냐에 따라서 Session-Parallel Mini-Batch이 굳이 필요하지 않을 수도 있습니다.

In [41]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [42]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,newTime,item_idx
5125,36,1292,5,1012707635,2002-02-03,0
5053,36,7,4,1012707697,2002-02-03,1
5170,36,1387,5,1015904819,2002-03-12,2
5267,36,1201,4,1015904819,2002-03-12,3
5122,36,1291,5,1015904836,2002-03-12,4
5123,36,2167,5,1015904905,2002-03-12,5
5290,36,2951,4,1015904905,2002-03-12,6
5359,36,2115,5,1015904905,2002-03-12,7
5073,36,1912,5,1015904924,2002-03-12,8
5113,36,2662,3,1015904924,2002-03-12,9


### SessionDataLoader

SessionDataset 객체를 받아서 Session-Parallel mini-batch를 만드는 클래스
* __iter__ 메소드는 모델 인풋, 라벨, 세션이 끝나는 곳의 위치를 yield
* mask는 후에 RNN Cell State를 초기화

In [43]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [44]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,newTime,item_idx
5125,36,1292,5,1012707635,2002-02-03,0
5053,36,7,4,1012707697,2002-02-03,1
5170,36,1387,5,1015904819,2002-03-12,2
5267,36,1201,4,1015904819,2002-03-12,3
5122,36,1291,5,1015904836,2002-03-12,4
5123,36,2167,5,1015904905,2002-03-12,5
5290,36,2951,4,1015904905,2002-03-12,6
5359,36,2115,5,1015904905,2002-03-12,7
5073,36,1912,5,1015904924,2002-03-12,8
5113,36,2662,3,1015904924,2002-03-12,9


In [45]:
iter_ex = iter(tr_data_loader)
iter_ex

<generator object SessionDataLoader.__iter__ at 0x7f647963bf50>

In [46]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 36 52 67]
Label Item Idx are :       [ 1 37 53 68]
Previous Masked Input Idx are []


### Step 3. 모델 구성
이 부분도 실습코드 내역을 참고하여 다양하게 모델 구조를 시도해볼 수 있습니다.
* 이번 자료에서는 MRR과 Recall@k를 사용하겠습니다. MRR은 정답 아이템이 나온 순번의 역수 값입니다.
* 따라서 정답 아이템이 추천 결과 앞쪽 순번에 나온다면 지표가 높아질 것이고 뒤쪽에 나오거나 안나온다면 지표가 낮아질 것입니다.

### 3.1 Evaluation Metric
* 모델 성능에 대한 지표로 precision이나 recall이 있음
  * Session-Based Recommendation Task에서는 모델이 k개의 아이템을 제시했을 때, 유저가 클릭/ 구매한 n개의 아이템이 많아야 좋습니다.
  * 이 때문에 recall의 개념을 확장한 recall@k 지표, precision의 개념을 확장한 Mean Average Precision@k 지표 등을 사용합니다.

* 순서에 민감한 지표인 MRR, NDCG 같은 지표도 사용
  * 추천에서는 몇 번째로 맞추느냐도 중요합니다. 구글에서 검색했을 때 1페이지에 원하는 결과가 나오지 않고 2페이지에 나온다면 유저 반응이 크게 떨어질 것입니다.

In [47]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

### 3.2 Model Architecture

* 모델 구조가 간단한 편이므로 Functional하게 모델 구현
* 학습 진행률을 모니터링하기 위해 사용하는 tqdm 라이브러리 설치
  * pip install tqdm

In [48]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [49]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

모델에 사용할 hyper-parameter를 class형식으로 관리

In [50]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=64, hsz=50, drop_rate=0.2, lr=0.001, epochs=20, k=20)

In [51]:
model = create_model(args)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 1, 2632)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      402600    
_________________________________________________________________
dropout (Dropout)            (64, 50)                  0         
_________________________________________________________________
dense (Dense)                (64, 2632)                134232    
Total params: 536,832
Trainable params: 536,832
Non-trainable params: 0
_________________________________________________________________


### 5.3 Model Training
지금까지 준비한 데이터셋과 모델을 통해 학습을 진행

배치 사이즈나 epoch 등의 설정은 위의 args에서 관리

In [52]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

### Step 4. 모델 학습
다양한 하이퍼파라미터를 변경해 보며 검증해 보도록 합니다. 실습코드에 언급되었던 Recall, MRR 등의 개념들도 함께 관리될 수 있도록 합니다.

In [53]:
train_model(model, args)

Train:   0%|          | 0/226 [00:00<?, ?it/s, accuracy=0, train_loss=7.24]     

	 - Recall@20 epoch 1: 0.071429
	 - MRR@20    epoch 1: 0.017605



Train:   0%|          | 0/226 [00:00<?, ?it/s, accuracy=0, train_loss=7.18]     

	 - Recall@20 epoch 2: 0.066964
	 - MRR@20    epoch 2: 0.011928



Train:   0%|          | 0/226 [00:00<?, ?it/s, accuracy=0, train_loss=7.15]     

...

	 - Recall@20 epoch 18: 0.052455
	 - MRR@20    epoch 18: 0.012838



Train:   0%|          | 0/226 [00:00<?, ?it/s, accuracy=0.0625, train_loss=6.17]

	 - Recall@20 epoch 19: 0.054688
	 - MRR@20    epoch 19: 0.014769



Evaluation:  26%|██▋       | 14/53 [00:03<00:10,  3.77it/s]

	 - Recall@20 epoch 20: 0.058036
	 - MRR@20    epoch 20: 0.015105



### Step 5. 모델 테스트
미리 구성한 테스트셋을 바탕으로 Recall, MRR 을 확인해 봅니다.

In [54]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  34%|███▍      | 45/131 [00:12<00:23,  3.71it/s]

	 - Recall@20: 0.041667
	 - MRR@20: 0.012544



args = Args(tr, val, test, batch_size=64, hsz=20, drop_rate=0.3, lr=0.001, epochs=20, k=20)
train, val = 30, 30

   	 - Recall@20: 0.045833
	 - MRR@20: 0.009747

train, val = 60, 60

	 - Recall@20: 0.042324
	 - MRR@20: 0.008055
     
     
args = Args(tr, val, test, batch_size=128, hsz=20, drop_rate=0.3, lr=0.001, epochs=20, k=20)


### feature의 선택
해당 영화를 개인이 다른 사람에게 추천해줄까를 예측하는 문제가 아닌 시스템이 어떠한 기준을 가지고 타인에게 영화를 추천해줄까의 문제입니다. 일반적으로 평점이라는 것은 개인의 취향에 따라 다르고, 영화의 특정 요소가 마음에 들지 않으면(예를 들어 배우의 연기, 클리셰, 결말 등) 개인의 평점은 낮아질 수도, 높아질 수도 있습니다. 그러나 시스템의 추천에 대한 내용을 판단하는 것이라면, 영화의 '선택'은 개인이 일관적으로 선호하는 요소를 가지고 있을 가능성이 높습니다(예를 들어 장르, 줄거리, 사전정보 등). 다시말해 개인이 선호하는 요소들을 군집화해 놓은 결과물이라고 해석할 수 있습니다. 따라서 평점 데이터는 활용하지 않았습니다. 